# Global Warming Level Timing Tools

This notebook explores the estimated timing of Global Warming Levels (GWLs) based on the IPCC's warming trajectory estimates from the AR6 report. This is the recommended method for estimating the timing of climate impacts on GWLs. The notebook contains descriptive information to help users understand the motivation and process for planning around GWLs, as well as demonstrations of tools for calculating ant visualizing GWL timing data. The approach illustrated in this notebook is also described in our guidance material [here](https://analytics.cal-adapt.org/data/about/).

**Intended application:** As a user, I want to understand how to understand how timing estimates for global warming levels are calculated, and use tools to translate target-years in to GWLs, or vice versa.

**Runtime:** Less than 5 minutes.

**Table of Contents**
1. Global Warming Levels and the hot model problem
2. Visualizing warming levels and IPCC scenario trajectories
3. A simple lookup tool for estimating GWL timing
4. Using GWL timing tools in a planning workflow


In [39]:
import pandas as pd
import numpy as np

from climakitae.util.warming_levels import get_gwl_at_year, get_year_at_gwl
from climakitaegui.explore.warming import IPCCVisualize


# 1. Global Warming Levels and the hot model problem

The **hot model problem** refers to the fact that several models from the latest generation of the Couple Model Intercomparison Project (CMIP6) are known to have a warming sensitivity to CO2 that is unrealistically high. While these models still produce excellent simulations of the global climate and how it will respond to continued warming, the **timing** of that warming signal is faster than the current scientific consensus. An accessible overview of the issue can be read [here](https://www.carbonbrief.org/guest-post-how-climate-scientists-should-handle-hot-models/).

Using a **Global Warming Levels** approach to studying climate change helps avoid the timing bias introduced by hot models in the following way:

* We use climate model data on global warming levels to answer the question: "What will regional impacts from climage change look like at 2.0°C of global warming?"
* We then use a different set of specialized simulations to answer the question: "When is the world most likely to hit 2.0°C of global warming under different emissions scenarios?"

The specialized models that are used to estimate the most likely warming timelines were developed as part of the IPCC's Sixth Assessment Report (AR6), and combine a variety of observational constraints on climate sensitivity with simplified global climate simulations. A full discussion of the sources included in these estimates can be found in [box 4.1 of the IPCC AR6 report](https://www.ipcc.ch/report/ar6/wg1/chapter/chapter-4/#box-4.1), and the data for these warming trajectories can be found [here](https://data.ceda.ac.uk/badc/ar6_wg1/data/spm/spm_08/v20210809/panel_a).


The resulting global warming trajectories can be visualized using this plotting tool:

In [40]:
gwl_viz = IPCCVisualize()
gwl_viz.plot_warming_trajectories()

:Overlay
   .Curve.Historical                      :Curve   [Year]   (Mean)
   .Area.I                                :Area   [Year]   (5%,95%)
   .Curve.SSP1_hyphen_minus_1_full_stop_9 :Curve   [Year]   (Mean)
   .Curve.SSP1_hyphen_minus_2_full_stop_6 :Curve   [Year]   (Mean)
   .Curve.SSP2_hyphen_minus_4_full_stop_5 :Curve   [Year]   (Mean)
   .Curve.SSP3_hyphen_minus_7_full_stop_0 :Curve   [Year]   (Mean)
   .Curve.SSP5_hyphen_minus_8_full_stop_5 :Curve   [Year]   (Mean)
   .HLine.I                               :HLine   [x,y]
   .Text.I                                :Text   [x,y]

NOTE: The climate scenarios for CMIP6 were defined in 2016, so for the purpose of all of the climate simulations discussed here and used throughout the Analytics Engine, the **"Historical" time period is defined as 1950-2014, and the future scenarios run from 2015-2100.**

# 2. Visualizing warming levels and IPCC scenario trajectories

This visualization tool has a few additional features to help illustate the timing of global warming levels.

In [41]:
gwl_viz.plot_warming_trajectories(ssp='SSP 3-7.0', warming_level=2.0)

:Overlay
   .Curve.Historical                      :Curve   [Year]   (Mean)
   .Area.I                                :Area   [Year]   (5%,95%)
   .Area.A_90_percent_interval            :Area   [Year]   (5%,95%)
   .Curve.SSP3_hyphen_minus_7_full_stop_0 :Curve   [Year]   (Mean)
   .VLine.I                               :VLine   [x,y]
   .Text.I                                :Text   [x,y]
   .VLine.II                              :VLine   [x,y]
   .VLine.III                             :VLine   [x,y]
   .Curve.I                               :Curve   [x]   (y)
   .Text.II                               :Text   [x,y]
   .HLine.I                               :HLine   [x,y]
   .Text.III                              :Text   [x,y]

If an individual SSP  scenario is specified, two additional elements are added to the visualization:

* 90% confidence interval -- shown as a shaded area, represents the **very likely** (90% confidence) range of uncertainty around the best estimate for this scenario's warming trajectory.
* Best estimate crossing year - indicated by the vertical dashed line, along with the range of possible crossing years corresponding to the 90% confidence interval.

In [42]:
gwl_viz.plot_warming_trajectories(ssp='SSP 2-4.5', warming_level=2.0)

:Overlay
   .Curve.Historical                      :Curve   [Year]   (Mean)
   .Area.I                                :Area   [Year]   (5%,95%)
   .Area.A_90_percent_interval            :Area   [Year]   (5%,95%)
   .Curve.SSP2_hyphen_minus_4_full_stop_5 :Curve   [Year]   (Mean)
   .VLine.I                               :VLine   [x,y]
   .Text.I                                :Text   [x,y]
   .VLine.II                              :VLine   [x,y]
   .VLine.III                             :VLine   [x,y]
   .Curve.I                               :Curve   [x]   (y)
   .Text.II                               :Text   [x,y]
   .HLine.I                               :HLine   [x,y]
   .Text.III                              :Text   [x,y]

Choosing a different scenario for the same warming level illustates how the best estimate for the **crossing year** and the **range of uncertainty** can change depending on the warming scenario.

In [43]:
gwl_viz.plot_warming_trajectories(ssp='SSP 1-2.6', warming_level=2.0)

:Overlay
   .Curve.Historical                      :Curve   [Year]   (Mean)
   .Area.I                                :Area   [Year]   (5%,95%)
   .Area.A_90_percent_interval            :Area   [Year]   (5%,95%)
   .Curve.SSP1_hyphen_minus_2_full_stop_6 :Curve   [Year]   (Mean)
   .HLine.I                               :HLine   [x,y]
   .Text.I                                :Text   [x,y]

Some warming levels are not reached by every scenario before the end of the century, so no best estimate year can be returned.

# 3. A simple lookup tool for estimating GWL timing

Two functions are available to easily estimate GWL timing using the IPCC's likely warming trajectories without the plotting visualization.

The ```get_year_at_gwl``` function returns the best estimate for the year a specified GWL is reached for each scenario, along with the years that represent a 90% confidence interval around that estimate, as illustrated above.

In [44]:
get_year_at_gwl(2.0)

,5%,Mean,95%
Historical,<NA>,<NA>,<NA>
SSP 1-1.9,<NA>,<NA>,<NA>
SSP 1-2.6,<NA>,<NA>,<NA>
SSP 2-4.5,2039,2053,2081
SSP 3-7.0,2037,2047,2061
SSP 5-8.5,2034,2042,2054


In [45]:
get_year_at_gwl(1.5, ssp='SSP 3-7.0')

,5%,Mean,95%
SSP 3-7.0,2024,2031,2041


Warming levels below approximatel 1.2°C are reached in the historical period (before 2015), and only return dates for the historical scenario.

In [46]:
get_year_at_gwl(0.8)

,5%,Mean,95%
Historical,1990,2002,2010
SSP 1-1.9,<NA>,<NA>,<NA>
SSP 1-2.6,<NA>,<NA>,<NA>
SSP 2-4.5,<NA>,<NA>,<NA>
SSP 3-7.0,<NA>,<NA>,<NA>
SSP 5-8.5,<NA>,<NA>,<NA>


In [47]:
get_year_at_gwl(1.0, ssp="Historical")

,5%,Mean,95%
Historical,2004,2010,<NA>


Note: Warming levels that fall near the edges of scenario definitions (including the transition between historical and future scenarios in 2015) may return NA values

The ```get_gwl_at_year``` function performs a lookup in the opposite direction from the same dataset, finding the *very likely* range of global warming levels for a specified future year and SSP scenario. This can be used to help choose which GWL targets are a good replacement for existing target-years in a planning process.

In [48]:
get_gwl_at_year(2040)

,5%,Mean,95%
SSP 1-1.9,1.27,1.55,1.86
SSP 1-2.6,1.34,1.62,1.96
SSP 2-4.5,1.45,1.71,2.05
SSP 3-7.0,1.50,1.78,2.13
SSP 5-8.5,1.59,1.93,2.31


In [49]:
get_gwl_at_year(2075, 'SSP 3-7.0')

,5%,Mean,95%
SSP 3-7.0,2.39,2.99,3.74


# Using GWL timing tools in a planning workflow

### Planning arounnd established Global Warming Level targets
The process for planning with global warming levels will look something like this:

1. Choose one or more target global warming levels
2. Use tools in the Analytics Engine to analyze regional climate impacts at each warming level
3. Choose an SSP scenario to inform timing estimates for global warming level targets
4. Use ```get_year_at_gwl``` to estimate the most likely range of years that the analyzed impacts will be realized

Using this process, which starts with specific GWL targets, decisions about which SSP scenario to use for timing estimates can be made and changed independant of the data analysis.




### Adapting target-year planning to a Global Warming Level Approach

If you are adapting a planning process that is based around specific target years to a global warming level framework, the process could look like this:

1. Start with a particular target year you would like to plan around (e.g. 2050)
2. Choose the SSP emissions scenario(s) you would like to use for estimating the timing of warming
3. Use the ```get_gwl_at_year``` tool to determine the most likely range of GWLs that will be reached at your target year
4. Use tools in the Analytics Engine to load data at the GWL(s) that correspond with your target year.

By analyzing impacts at multiple GWLs that span the likely range from the IPCC trajectories, this analysis can capture a range of uncertainty around the target year.


Note: In this planning process, the SSP that is used to translate years into GWLs must be chosen *before* the analysis is performed, and can not be changed afterwards. 

